In [151]:
import numpy as np

# Constants

W_P = 11.5
W_S = 13

S = 420 # ft^2 - from P vs. S plot
P = 470 # hp - from P vs. S plot

# Start with a weight estimate
W0 = 8752

A = 0.74  # Regression constants from Martin's Metabook which cites it from Raymer (2006, Table 3.1)
C = -0.03

c_sl = 0.15 / 3600 # 1/s - specific fuel consumtion factor in lb / hp-hr # float(input("Enter the value of c_SL: "))  # Algorithm 3 pg. 50 Metabook
S_wet_initial = 1458 # is this accurate still?

c_cr = 0.15 / 3600
weight_crew = 180
weight_payload = 2000

V = 250 * 1.688

rho = 0.00238
rho_condition = 0.0023672

cL_max_takeoff = 1.7 # CHECK #float(input("Enter the cL_max of the wing: ")) #Needs to be approximated based on historical data
cL_max_cruise = 1.6 # CHECK
cL_max_landing = 1.8 # CHECK
V_stall = 52 * 1.688# ft/s approximation - stall speed of a small aircraft # math.sqrt((2 * W_0) / (rho * S_ref * cL_max)) #float(input("Enter the stall velocity (ft/s): "))

s_to = 1500 # RFP #float(input("Enter the takeoff distance (ft): "))
a = 0.0149 
b = 8.134 

s_a = 600 # ft - from Raymer's text pg. 138
s_landing = 1500 # in ft

eta_p = 0.95
c = 1.0447 # Source - Gama
d = 0.5326 # Source - Gama
c_f = 0.0055 # figure 4.4 on Metabook - "light aircraft - single engine"
G3 = 0.083
G1 = 0.03

n = 2 # float(input("Enter the load factor (#g): "))

eff_clean = 0.825



In [152]:
# Starter equations
P_guess = W0 / W_P
S_guess = W0 / W_S

S_wet_initial_rest = S_wet_initial - (2*60*7)

discriminant = b**2 - 4*a*(-s_to)
top23 = (-b + np.sqrt(discriminant)) / (2*a)

V_climb = V_stall * 1.1
q_climb = 0.5 * rho * (V_climb)**2

V_balked = V_stall * 1.15
q_balked = 0.5 * rho * (V_balked)**2

V_mn = V * 0.5
q_maneuver = 0.5 * rho_condition * V_mn**2

v_cruise = V * 0.5
q_cruise = 0.5 * rho_condition * v_cruise**2

V_ceiling = V * 0.5
Gceiling = (100 / 60) / V_ceiling
q_ceiling = 0.5 * rho * (V_ceiling)**2



In [153]:
# Component Weights

# For more accurate values, using equations provided by Raymer's text pg. 609

# weight of the wing

S_w = 420 # ft^2 - wing area (coming from P vs S plot)
W_fw = 1350 # lbs - weight of the fuel in wings
AR = 8.57 # ft - aspect ratio of the wing
sweep_angle_w = np.deg2rad(2) # wing sweep angle at 25% MAC

rho = 0.00238 # slugs/ft^3 - rho is the seal level standard value - Raymer's pg. 131
V = 250 * 1.688
v_cruise = V * 0.5
q = 0.5 * rho * v_cruise**2 # dynamic pressure at cruise
Lambda = 1 # taper ratio of the wing
t_c =  0.12 # thickness to chord ratio - depends on the type of airfoil
Nz = 4.725 # ultimate load factor = 1.5 * limit load factor = 1.5 * 3.15 Table 14.2 on pg. 524 on Raymer
Wdg = 7067 # flight design gross weight 
Scsw = 425 # control surface area (wing-mounted, includes flaps)
Bw = 60 # wing span

W_wing = (0.036 * (S_w**0.758) * (W_fw ** 0.0035) *
          ((A / (np.cos(sweep_angle_w) ** 2))**0.6) * (q ** 0.006) *
          (Lambda ** (0.04)) * (((100 * t_c) / (np.cos(sweep_angle_w))) ** (-0.3)) *
          ((Nz * Wdg) ** 0.49))

fudge_factor_w = 0.85 # from table 15.4 pg. 614 Raymers
print("Weight of the wing (lbs): " + str(fudge_factor_w * W_wing))

# weight of horizontal tail
Sht = 89 # ft^2 - horizontal tail area # NEED TO UPDATE 
sweep_angle_ht = 0 # sweep angle of the horizontal tail
Lambda_ht = 1 # taper ratio of horizontal tail
Kuht = 1 # 1.143 for unit (all-moving) horizontal tail; = 1 otherwise
Fw = 4.5 # ft, fuselage width at horizontal tail intersection 
Bh = 19 # ft, horizontal tail span
Lt = 18.375 # ft - tail length from wing quarter MAC to tail quarter MAC
Ky = 0.3 * Lt # aircraft pitching radius of gyration (about 0.3*Lt)
Ah = 4 # 19/4.68
Se = 6 # elevator area ft^2

W_ht = (0.016 * ((Nz * Wdg) ** 0.414) * (q ** 0.168) *
        (Sht ** 0.896) * (((100 * t_c) / (np.cos(sweep_angle_ht))) ** (-0.12)) *
        ((AR / ((np.cos(sweep_angle_ht)) ** 2)) ** 0.043) * (Lambda_ht ** (-0.02))
        )

fudge_factor_t = 0.83
print("Weight of the horizontal tail (lbs): " + str(fudge_factor_t * W_ht))

# weight of vertical tail
Ht_Hv = 0 # for conventional tail config Ht/Hv = 0 and 1 for T tail
Svt = 55 # ft^2 - vertical tail area
sweep_angle_vt = np.deg2rad(20)
Lambda_vt = 0.43 # taper ratio of vertical tail
Kz = Lt # aircraft yawing radius of gyration (about Lt)
Av = 11/5 # 11 / ((7 + 3)/2)

W_vt = (0.073 * (1 + (0.2 * Ht_Hv)) * ((Nz * Wdg) ** 0.376) *
        (q ** 0.122) * (Svt ** 0.873) * (((100 * t_c) / (np.cos(sweep_angle_ht))) ** (-0.49)) *
        ((A / ((np.cos(Lambda_vt))**2))**0.357) * (Lambda_vt ** 0.039))

print("Weight of the vertical tail (lbs): " + str(fudge_factor_t * W_vt))

# Fuselage weight
Sf = 200 # ft^2 - fuselage wetted area _ NEED REAL VALUE
L = 35 # fuselage structural length
D = 6 # fuselage structural depth

V_pr = 280 # volume of pressurized section 8 (tall) x 7 (wide) x 5 (depth)
P_delta = 8 # psi - cabin pressure differential usually 8 psi
W_press = 11.9 * (V_pr * P_delta) ** 0.271 # weight penalty due to pressurization

Kdoor = 1 # 1.0 if no cargo door; - CHECK
# 1.06 if one side cargo door; 
# 1.12 if two side cargo doors; 
# 1.12 if aft clamshell door; 
# 1.25 if two side cargo doors and aft clamshell door

Kws = 0.75 * ((1 + (2 * Lambda)) / (1 + Lambda)) * (Bw / L) * (np.tan(sweep_angle_w)) # correction factor
KLg = 1.12 # 1.12 if fuselage-mounted main landing gear; 1.0 otherwise

W_fuselage = (0.052 * (Sf ** 1.086) * ((Nz * Wdg) ** 0.177) * (Lt ** (-0.051)) *
              ((L/D) ** -0.072) * (q ** 0.241)) + W_press

fudge_factor_f = 1.2 # for carrier-based aircraft
print("Weight of the fuselage (lbs): " + str(fudge_factor_f * W_fuselage))

# Main and tail landing gear weight
Nl = 1.5 * 3 # landing load factor = Ngear * 1.5 ; Ngear can be found on pg. 371 of Raymer
Wl = 6784 # landing design gross weight NEED TO CHECK for at602 takeoff: 12500 landing: 12000 so 12000/12500 = 0.96*7067 
Lm = 29.5 # for AT602 extended length of the main landing gear (in) - NEED TO CHECK
Ln = 33.5 # for AT602 extended length of the nose landing gear (in) - NEED TO CHECK
Kmp = 1 # 1.126 for kneeling gear otherwise 1
Nmw = 2 # number of main wheels
Nnw = 2 # number of nosewheels
Nmss = 2 # main gear shock struts 
V_stall = 52 * 1.688 # ft/s
Knp = 1 # 1.15 for kneeling gear otherwise 1

W_main_lg = ( 0.095 * ((Nl * Wl) ** 0.768) * ((Lm/12)**0.409))

W_tail_lg = 0.125 * ((Nl * Wl) ** 0.566) * ((Ln / 12)**0.845)

W_total_lg = W_main_lg + W_tail_lg - (0.014 * Wdg) # because landing gear is nonretractable

fudge_factor_lg = 1.2 # for carrier-based aircraft
print("Weight of the landing gear (lbs): " + str(fudge_factor_lg * W_total_lg))

# Installed engine total weight

W_en = 400 # engine weight for pt6 - using approx. 400 - NEED TO CHECK

Kng = 1 # 1.017 for pylon-mounted nacelle; = 1 otherwise
NLt = 6 # ft. nacelle length - CHECK

Kp = 1.4 # for engine with propeller or 1 otherwise
Ktr = 1 # 1.18 for jet with thrust reverser or 1 otherwise - CHECK

Wec = (2.331 * (W_en ** 0.901) * Kp * Ktr) # lbs, weight of engine and contents 
Nw = 2 # nacelle width ft - CHECK
N_en = 1 # number of engines 
Sn = 0 # nacelle wetted area ft^2
W_ie = 2.575 * (W_en ** 0.922) * N_en # includes propeller and engine mounts

print("Weight of the installed engine total (lbs): " + str(W_ie))

# Engine controls weight 

Lec = 20 # routing distance from engine front to cockpit, ft - CHECK
W_eng_con = (5 * N_en) + (0.8 * Lec)

print("Weight of the engine controls (lbs): " + str(W_eng_con))

# Weight of starter (pneumatic)

W_sp = 49.19 * (((N_en * W_en) / 1000) ** 0.541)

print("Weight of pneumatics (lbs): " + str(W_sp))

# Fuel system weight

Vt = 171 # gal - total fuel volume - CHECK (coverted 1350 lbs to gallon)
Vi = 90 # gal - integral tanks volume - CHECK (171/2 = 85.5)
Nt = 2 # number of fuel tanks - CHECK
Vp = Vi # self-sealing "protected" tanks volume, gal - CHECK

W_fuel_sys = (
    2.49 * (Vt ** 0.726) * ((1 / (1 + (Vi/Vt))) ** 0.363) * (Nt ** 0.242) *
    (N_en ** 0.157)
)

W_fuel = 1350 # lbs

print("Weight of the fuel system (lbs): " + str(W_fuel_sys))

# Flight controls weight

Nf = 5 # number of separate functions performed by 
# surface controls, including rudder, aileron, elevator, ﬂaps, spoiler, 
# and speed brakes (typically 4 – 7) - CHECK
Nm = 2 # number of surface controls driven by 
# mechanical actuation instead of hydraulics (must be N f and is typically 0– 3) - CHECK
Scs =  (2 * 9) + (1 * 6) + (2.1 * 9) + (1.7 * 14) # ft^2, total area of control surfaces 
Iyaw = (1/12) * (Wdg) * ((40**2) + (5**2)) # yawing moment of inertia, lb-ft^2, estimated using this formula Lz = (1/12) * (total mass) * ((length of aircraft)^2 + (width of aircraft)^2)

W_flight_controls = (
    (0.053 * (L ** 1.536) * (Bw ** 0.371) * (((Nz * Wdg) * 10** (-4))**0.80))
)

print("Weight of flight controls (lbs): " + str(W_flight_controls))

# Weight of instruments 

Kr = 1 # 1.133 for reciprocating engine; 1 otherwise
Ktp = 0.793 # for turboprop; 1 otherwise
Nc = 1 # number of crew

W_instruments = (4.509 * Kr * Ktp * (Nc**0.541) *
                 N_en * ((L + Bw) ** 0.5))

print("Weight of flight instruments (lbs): " + str(W_instruments))


# Hydraulics Weight

Kh = 0.11 # 0.05 for low subsonic with hydraulics for brakes and retracts only; 
# 0.11 for medium subsonic with hydraulics for ﬂaps;
# 0.12 for high subsonic with hydraulic ﬂight controls;
# 0.013 for light plane with hydraulic brakes only (and use M = 0.1) - CHECK
M = 0.195 # Mach number (design maximum) 250 knots / 640 knots = 0.39
W_hydraulics = Kh * (Wdg ** 0.8) * (M ** 0.5)
print("Weight of flight hydraulics (lbs): " + str(W_hydraulics))

# Avionics weight

Wuav = 1000 # lbs - uninstalled avionics weight CHECK
W_avionics = 2.117 * (Wuav ** 0.933)
print("Weight of avionics (lbs): " + str(W_avionics))

# Electrical weight

W_electrical = 12.57 * (W_fuel_sys + W_avionics) ** 0.51
print("Weight of electrical components (lbs): " + str(W_electrical))

# Furnishing 

Wc = 2000 # maximum cargo weight
W_furnishings = (0.0582 * Wdg) - 65
print("Weight of furnishings (lbs): " + str(W_furnishings))

# Air conditioning & Anti-ice

Np = 1 # number of people on board
W_ac_ai = (
    0.265 * (Wdg ** 0.52) * (Np ** 0.68) * (W_avionics ** 0.17) * (M ** 0.08)
)
print("Weight of air conditioning and anti-ice (lbs): " + str(W_ac_ai))



Weight of the wing (lbs): 204.1973708707792
Weight of the horizontal tail (lbs): 87.68078303213291
Weight of the vertical tail (lbs): 44.95582368898181
Weight of the fuselage (lbs): 361.37029212851746
Weight of the landing gear (lbs): 462.7005400137367
Weight of the installed engine total (lbs): 645.4707183716508
Weight of the engine controls (lbs): 21.0
Weight of pneumatics (lbs): 29.96341282454805
Weight of the fuel system (lbs): 105.57008375333591
Weight of flight controls (lbs): 149.47481443638802
Weight of flight instruments (lbs): 34.85099849068969
Weight of flight hydraulics (lbs): 58.317387990454755
Weight of avionics (lbs): 1332.6645891086878
Weight of electrical components (lbs): 512.6588423249084
Weight of furnishings (lbs): 346.2994
Weight of air conditioning and anti-ice (lbs): 79.2963936254092


In [154]:

weight_components = (W_wing + W_ht + W_vt + W_fuselage + W_total_lg + 
                     W_ie + W_eng_con + W_sp + W_fuel_sys + W_flight_controls + 
                     W_instruments + W_hydraulics + W_avionics + W_electrical + 
                     W_furnishings + W_ac_ai + W_fuel)

In [155]:
# Functions

def weight_exp(W0):
    weight = A * ((W0**C))
    return weight

In [156]:
# Calcaute your fuel weight for each flight phase using this weight 

weight_tol = 1e-6
tol = 2e-6

while tol > weight_tol:
    We_W0 = weight_exp(W0)
    #We = We_W0 * W0
    #density_wing = 2.5  # lb/ft^2 Metabook pg. 76
    S_initial = W0/W_S
    P_initial = W0/W_P
    #We += density_wing * (S - S_initial)
    #W_engine_dp = (P_initial ** 0.9306) * (10 ** -0.1205) # Metabook 79
    #W_engine = (P ** 0.9306) * (10 ** -0.1205)
    #We += (W_engine) - (W_engine_dp)

    # Algorithm 3 - fuel fraction calculation
    T = P * 550 / (V_climb)
    W1_W0 = 1 - c_sl * (15 / 60) * (0.05 * T / W0)  # fuel burned from running the engine for 15 min at 5% max
    W1 = W1_W0 * W0
    W2_W1 = 1 - c_sl * (1 / 60) * (T / W1)  # Fuel burned from running the engine for 1 min at max

    # Climb segment - breaking it into 10 segments for better approximations

    steps = 20
    h_start = 0 
    h_end = 583
    c_cl =  0.2 / 3600 # 1 / s - assumed 
    g = 32.2 # ft/s^2

    T_W = (1/W_P) * ((550 * eta_p) / V_climb)

    h_values = np.linspace(h_start, h_end, steps)
    W2 = W2_W1 * W1 # starting with weight from previous weight 
    x_climb = 0 # total range covered during climb 

    #print(W2)

    W3 = W2
    #print(W2)

    for i in range(1, steps):

        # Best climb velocity based on slide 58 lecb02

        T_W_i = T / W3
        #termv_1 = T_W_i
        #termv_1 = np.sqrt(T_W_i**2 + (12*CD_0*k))
        #termv_2 = (T/W3) + termv_1
        #termv_3 = np.sqrt(((W3/S) / (3*rho*CD_0)) * termv_2)

        #V_climb_new[i] = termv_3
        #V_climb_delta = V_climb_new[i] - V_climb_new[i-1]
        #print(V_climb)

        delta_h = h_values[i] - h_values[i - 1]
        delta_he = delta_h # the rest of it cancels out since its delta + #(V_climb_delta**2 / (2 * g))

        gamma = np.deg2rad(10) # flight path angle (around 8 - 15 degrees)
        dh_dt = V_climb * np.sin(gamma)
        # dh_dt = dhe/dt = V(1 - (D/T)) -> 1 - (D/T) = (dhe/dt)/V # slide 57

        one_minus_DT = np.sin(gamma)

        weight_frac_1 = np.exp((-c_cl * delta_he * T_W) / (eta_p * one_minus_DT))
        W3 = W3 * weight_frac_1
        #W2 = W3


        # dh_dt = dhe/dt = V(1 - (D/T)) -> T - D = (T/V) * (dhe/dt)
        Ps = V_climb * ((T/V_climb) * dh_dt) / W3 # power per segment - slide 59 lec b02

        x_climb = x_climb + ((delta_he * V_climb) / Ps) # in ft - coming out around 57100 ft which is about 9.3974 nmi

    # W3_W2 = 0.985  # climb segment in Algorithm 3 - using historical values

    # Cruise segment - updated for multiple segments approach 
    R_start = 0
    R_end = 600 # nmi - CHECK
    x_climb = x_climb / 6076 # nmi
    R = R_end - x_climb
    delta_R = R / steps
    CD_0 = c_f * (S_wet_initial_rest + (2 * S)) / S  # Zero-lift drag coefficient
    AR = (60**2)/S  # Aspect ratio NEEDS TO CHANGE FOR CHANGE OF AIRCRAFT - changed
    k = 1 / (np.pi * eff_clean * AR)  # Induced drag factor
    rho_term1 = (2 * W3) / ((v_cruise ** 2) * S) # slide 30 lec b02
    rho_term2 = np.sqrt(k / CD_0) # slide 30 lec b02
    rho_new = rho_term1 * rho_term2 # desnity at the start of cruise 
    W4 = W3 # starting weight for this section

    #print(W3) 

    for i in range(0, steps):
        C_L = (2 * W4) / (rho_new * (v_cruise**2) * S)
        L_D = C_L / (CD_0 + (k * C_L**2))

        term1 = delta_R * c_cr
        term2 = v_cruise * L_D
        weight_frac = np.exp(-term1/term2)
        W4 = W4 * weight_frac
        #W3 = W4
   # c_f = 0.0055 # figure 4.4 on Metabook - "light aircraft - single engine"
    #eff_clean = 0.825  # Oswald efficiency factor

    #CL = np.sqrt(CD_0 / k)  # Optimal CL for max L/D (Fix applied)
    #L_D = (0.94 * CL) / (CD_0 + k * CL**2)  # Lift-to-drag ratio
            
    #W4_W3 = np.exp(-(600 * 1.151) * c_cr / (125 * 1.151 * L_D))  # weight fraction during cruising

    # Loitering segnment - updated - no segnments needed as we will be flying at (L/D)max slide 60 lecb02
    E = 1.5 # endurance - set to 1.5 hrs
    term3 = E * v_cruise * c_cr
    CL = np.sqrt(CD_0/k)
    CD = CD_0 + (k * (CL**2))
    LD = CL/CD
    term4 = eta_p * (LD)
            
    W5_W4 = np.exp(-term3/term4)

    W5 = W4 * W5_W4

    W6_W5 = 0.99  # Descent historical value
    W6 = W5 * W6_W5

    W7_W6 = 0.995  # Landing historical value
    W7 = W6 * W7_W6

    W7_W0 = W7_W6 * W6_W5 * W5_W4 * (W4/W3) * (W3/W2) * W2_W1 * W1_W0

    Wf_W0 = 1 - W7_W0 
    #We_W0 = A * (W0 ** C)
    #We = 5222 #lbs - empty weight from RFP

    Wf = W0 * Wf_W0
    We = 5222 # W0 * We_W0 - from PDR
    We_W0 = We/W0
    
    W0_new = (weight_components + weight_crew + weight_payload) /  (1 - Wf_W0 - (We_W0)) 
    # this bottom part ((-1 + Wf_W0 + (We_W0))) is made to be negative just for the numbers to work out 
    # it is actually supposed to be (1 - Wf_W0 - (We_W0)) - need to still work this out
              #W1 + W2 + W3 + W4 + W5 + W6 + W7)  # calculating the new takeoff gross weight
    newtol = abs(W0_new - W0) / abs(W0_new)  # finding the difference between the last and current iteration of W0
    tol = newtol
    W0 = W0_new  # updating the takeoff gross weight
print(W0)
#W0_S0 = W0_new / S
#S_wet = (10 ** c) * (W0_new ** d)# 4605 * 142.45 # float(input("Enter the reference wing wet area (ft^2): "))
#S_wet_S_ref =  S_wet / S # 4 - from Raymer's text
#parasite_drag_coeff = c_f * S_wet_S_ref

14658.559579847026
